In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient
import pandas as pd

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [3]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [4]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [5]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [6]:
artists = list(s.find({}, {'artist_id':1, '_id':0}))
artists = [int(x['artist_id']) for x in artists]

writers = list(s.find({}, {'writers':1, '_id':0}))
for writer in writers:
    if writer:
        for w in writer['writers']:
            artists.append(int(w['id']))
            
featured_artists = list(s.find({}, {'featured_artists':1, '_id':0}))
for fa in featured_artists:
    if fa:
        for w in fa['featured_artists']:
            artists.append(int(w['id']))

producers = list(s.find({}, {'producers':1, '_id':0}))
for p in producers:
    if p:
        for w in p['producers']:
            artists.append(int(w['id']))

desc_refs = list(s.find({}, {'description_refs.artist':1, 'description_refs.song':1, '_id':0}))
for dr in desc_refs:
    if 'artist' in dr['description_refs']:
        a = dr['description_refs']['artist']
        for artist in a:
            artists.append(int(artist['id']))
    if 'song' in dr['description_refs']:
        pass
    
anno_refs = list(s.find({}, {'references.artist':1, 'references.song':1, '_id':0}))
for ar in anno_refs:
    if 'artist' in ar['references']:
        a = ar['references']['artist']
        for artist in a:
            artists.append(int(artist['id']))
    if 'song' in ar['references']:
        pass
    
sampled_songs = list(s.find({}, {'sampled_songs':1, '_id':0}))
for samp in sampled_songs:
    if samp:
        for w in samp:
            for song in samp[w]:
                artists.append(song['primary_artist']['id'])

print "before uniquing: ", len(artists)
artists = [int(x) for x in artists]
artists = list(set(artists))
print "after uniquing: ", len(artists)

before uniquing:  1163
after uniquing:  452


In [7]:
weights = {'artist_id': 10., 'producers':10., 'writers': 10., 'featured_artists': 8., \
          'sampled_songs': 7., 'description_refs': 5., 'references': 4.}

In [8]:
def get_info(song, weights=weights, artists=artists):
    contributors = defaultdict(list)
    cont_weights = defaultdict(float)
    total = 0
    info = s.find({'id':song}, {'_id':0, 'description_refs.artist':1, 'description_refs.song':1, \
                       'sampled_songs':1, 'producers':1, 'writers':1, \
                       'featured_artists':1, 'artist_id':1, 'id':1}).next()
    for e in info:
        if e == 'artist_id':
            contributors['artist_id'].append(int(info[e]))
        if e == 'description_refs':
            if 'artist' in e:
                for a in e['artist']:
                    contributors['description_refs'].append(int(a['id']))
        if e == 'references' in info:
            if 'artist' in e:
                for a in e['artist']:
                    contributors['references'].append(int(a['id']))
        if e == 'producers':
            for a in info[e]:
                contributors['producers'].append(int(a['id']))
        if e == 'writers':
            for a in info[e]:
                contributors['writers'].append(int(a['id']))
        if e == 'featured_artists':
            for a in info[e]:
                contributors['featured_artists'].append(int(a['id']))
        if e == 'sampled_songs':
            for samp in info[e]:
                contributors['sampled_songs'].append(int(samp['primary_artist']['id']))
    for artist_type in contributors:
        total += weights[artist_type]
    for artist_type, influencers in contributors.iteritems():
        frac = weights[artist_type]/len(influencers)
        for influencer in influencers:
            cont_weights[influencer] += frac/total
    influences = {(a):(0 if a not in cont_weights.keys() else cont_weights[a]) for a in artists}
    influences_series = pd.Series(data=influences.values(), index=influences.keys())
    return influences_series

In [9]:
songs = [int(x['id']) for x in list(s.find({'id':{'$exists':1}}, {'id':1, '_id':0}))]

In [10]:
df = pd.DataFrame(columns = songs)

In [11]:
df.shape

(0, 232)

In [12]:
for song in songs:
    try:
        series = get_info(song)
        df[song] = series
    except:
        print "unable to score songid: ", song
        continue

In [13]:
for col in df:
    thesum = df[col].sum()
    if thesum<.99999 or thesum>1.00001:
        print col, thesum

In [14]:
df.shape

(452, 232)

Columns are song ids and rows are influencers (artist ids). There are 452 unique artists and 232 unique songs at this point in time. The sum of any given column should be 1 because the influence scores are normalized.

In [15]:
df

,653082,2396122,2415289,2369079,2411389,2304247,2396302,599451,2398213,2433137,...,217270,1800164,47702,58349,73198,2045,517621,164344,639914,505193
1,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
2,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
3,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
4,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.285714,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
2057,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
329739,0,0,0,0.0,0,0.000000,0,0.0,0.037594,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
13,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
589827,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
20503,0,0,0,0.5,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0
589828,0,0,0,0.0,0,0.000000,0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0.000000,0.000000,0.0


The below code is a little sanity check. By seeing column for songid = 639914 in the above df, we see artistid of 1362 is .1111. (1/9) This validates that.

In [75]:
# first edit the get_info def to return contributors dict
# info = get_info(639914)

print info

#do the rest by hand 
total = 0
cont_weights = defaultdict(float)
for artist_type in info:
    total += weights[artist_type]
for artist_type, influencers in info.iteritems():
    frac = weights[artist_type]/len(influencers)
    for influencer in influencers:
        cont_weights[influencer] += frac/total
        if influencer == 1362:
            print influencer, artist_type, frac, total, frac/total

print cont_weights

defaultdict(<type 'list'>, {'producers': [574528, 290415], 'artist_id': [2358], 'writers': [574528, 290415, 1362]})
1362 writers 3.33333333333 30.0 0.111111111111
defaultdict(<type 'float'>, {574528: 0.2777777777777778, 1362: 0.11111111111111112, 2358: 0.3333333333333333, 290415: 0.2777777777777778})
